# スキャルピング条件の改善分析

## 目的
- バックテスト結果から「儲かった銘柄」と「損した銘柄」の特徴を比較
- どのテクニカル指標が利益と相関しているか分析
- 現状のスコアリング配点の妥当性を検証
- 改善すべき指標と優先順位を特定

In [12]:
import sys
from pathlib import Path

ROOT = Path.cwd().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

# matplotlib日本語対応
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Hiragino Sans', 'Yu Gothic']
plt.rcParams['axes.unicode_minus'] = False

## 1. データ読み込み

In [13]:
from common_cfg.paths import PARQUET_DIR

TEST_DIR = PARQUET_DIR / "test"
SCORED_PATH = TEST_DIR / "mock_screened_100stocks_scored.parquet"

df = pd.read_parquet(SCORED_PATH)

print(f"Total rows: {len(df):,}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"\nColumns: {list(df.columns)}")

Total rows: 3,975
Date range: 2025-08-21 00:00:00 to 2025-10-20 00:00:00

Columns: ['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'ticker', 'prevClose', 'change_pct', 'tr', 'atr14', 'atr14_pct', 'ma5', 'ma25', 'rsi14', 'vol_ma10', 'vol_ratio', 'overall_rating', 'stock_name', 'market', 'sectors', 'series', 'topixnewindexseries', 'entry_filter_passed', 'entry_score', 'entry_rank', 'active_filter_passed', 'active_score', 'active_rank']


## 2. 翌日の値動きを計算

In [14]:
# 営業日でソート
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

# 翌日のOpen/Closeを取得（銘柄ごと）
df['next_Open'] = df.groupby('ticker')['Open'].shift(-1)
df['next_Close'] = df.groupby('ticker')['Close'].shift(-1)

# 翌日の値動き（Open→Close）
df['next_day_return'] = ((df['next_Close'] - df['next_Open']) / df['next_Open'] * 100)

# 翌日のデータがある行のみ
df_with_next = df[df['next_Open'].notna()].copy()

print(f"翌日データあり: {len(df_with_next):,} rows")
print(f"\nnext_day_return の統計:")
print(df_with_next['next_day_return'].describe())

翌日データあり: 3,864 rows

next_day_return の統計:
count    3864.000000
mean       -0.158433
std         2.173878
min       -20.631787
25%        -0.909091
50%        -0.116896
75%         0.529356
max        27.516779
Name: next_day_return, dtype: float64


## 3. Entry候補の分析

In [15]:
# Entry条件を満たした銘柄
df_entry = df_with_next[df_with_next['entry_filter_passed']].copy()

print(f"Entry候補: {len(df_entry)} rows")
print(f"\n翌日リターンの統計:")
print(df_entry['next_day_return'].describe())

# 勝率
win_rate = (df_entry['next_day_return'] > 0).sum() / len(df_entry) * 100
print(f"\n翌日プラスになる確率: {win_rate:.1f}%")
print(f"翌日マイナスになる確率: {100 - win_rate:.1f}%")

Entry候補: 203 rows

翌日リターンの統計:
count    203.000000
mean      -0.091932
std        1.278056
min       -2.682927
25%       -0.777988
50%       -0.178891
75%        0.440208
max        8.969072
Name: next_day_return, dtype: float64

翌日プラスになる確率: 39.9%
翌日マイナスになる確率: 60.1%


### 3.1 Entry: 儲かった vs 損した の比較

In [16]:
# 儲かった銘柄 vs 損した銘柄
df_entry_win = df_entry[df_entry['next_day_return'] > 0].copy()
df_entry_lose = df_entry[df_entry['next_day_return'] <= 0].copy()

print(f"儲かった: {len(df_entry_win)} ({len(df_entry_win)/len(df_entry)*100:.1f}%)")
print(f"損した: {len(df_entry_lose)} ({len(df_entry_lose)/len(df_entry)*100:.1f}%)")

# テクニカル指標の平均値を比較
indicators = ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'entry_score']

comparison = pd.DataFrame({
    '指標': indicators,
    '儲かった平均': [df_entry_win[ind].mean() for ind in indicators],
    '損した平均': [df_entry_lose[ind].mean() for ind in indicators],
})
comparison['差分'] = comparison['儲かった平均'] - comparison['損した平均']
comparison['差分%'] = (comparison['差分'] / comparison['損した平均'].abs() * 100)

print("\n--- Entry: 儲かった vs 損した の指標比較 ---")
display(comparison.style.set_properties(**{'text-align': 'left'}))

儲かった: 81 (39.9%)
損した: 122 (60.1%)

--- Entry: 儲かった vs 損した の指標比較 ---


,指標,儲かった平均,損した平均,差分,差分%
0,change_pct,-0.207407,0.043279,-0.250686,-579.236813
1,atr14_pct,2.067037,2.038033,0.029004,1.423149
2,rsi14,46.619136,56.715410,-10.096274,-17.801642
3,vol_ratio,101.458642,104.166148,-2.707506,-2.599218
4,entry_score,59.098765,58.131148,0.967618,1.664543


### 3.2 Entry: overall_rating の効果

In [17]:
# overall_rating別の翌日リターン
rating_analysis = df_entry.groupby('overall_rating').agg({
    'next_day_return': ['count', 'mean', 'median'],
    'ticker': 'count'
}).round(2)

rating_analysis.columns = ['件数', '平均リターン%', '中央値リターン%', '銘柄数']
rating_analysis['勝率%'] = df_entry.groupby('overall_rating').apply(
    lambda x: (x['next_day_return'] > 0).sum() / len(x) * 100
).round(1)

print("--- Entry: overall_rating 別の翌日パフォーマンス ---")
display(rating_analysis.style.set_properties(**{'text-align': 'left'}))

print("\n⚠️ overall_rating が '中立' だけなら、この指標は無意味")

--- Entry: overall_rating 別の翌日パフォーマンス ---


,件数,平均リターン%,中央値リターン%,銘柄数,勝率%
overall_rating,,,,,
中立,203,-0.090000,-0.180000,203,39.900000



⚠️ overall_rating が '中立' だけなら、この指標は無意味


### 3.3 Entry: 各指標と翌日リターンの相関分析

In [18]:
# 相関係数を計算
correlation_indicators = ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'entry_score']
correlations = {}

for ind in correlation_indicators:
    corr = df_entry[[ind, 'next_day_return']].corr().iloc[0, 1]
    correlations[ind] = corr

corr_df = pd.DataFrame({
    '指標': list(correlations.keys()),
    '相関係数': list(correlations.values())
}).sort_values('相関係数', key=abs, ascending=False)

print("--- Entry: 各指標と翌日リターンの相関係数 ---")
display(corr_df.style.set_properties(**{'text-align': 'left'}))

print("\n解釈:")
print("- 相関係数 > 0.1: その指標が高いほど翌日儲かる")
print("- 相関係数 < -0.1: その指標が高いほど翌日損する")
print("- |相関係数| < 0.1: ほぼ関係ない")

--- Entry: 各指標と翌日リターンの相関係数 ---


,指標,相関係数
2,rsi14,-0.116017
3,vol_ratio,-0.081540
1,atr14_pct,0.079886
4,entry_score,0.014702
0,change_pct,0.001511



解釈:
- 相関係数 > 0.1: その指標が高いほど翌日儲かる
- 相関係数 < -0.1: その指標が高いほど翌日損する
- |相関係数| < 0.1: ほぼ関係ない


## 4. Active候補の分析

In [19]:
# Active条件を満たした銘柄
df_active = df_with_next[df_with_next['active_filter_passed']].copy()

print(f"Active候補: {len(df_active)} rows")
print(f"\n翌日リターンの統計:")
print(df_active['next_day_return'].describe())

# 勝率
win_rate = (df_active['next_day_return'] > 0).sum() / len(df_active) * 100
print(f"\n翌日プラスになる確率: {win_rate:.1f}%")
print(f"翌日マイナスになる確率: {100 - win_rate:.1f}%")

Active候補: 342 rows

翌日リターンの統計:
count    342.000000
mean      -0.764673
std        3.701964
min      -15.428571
25%       -2.777996
50%       -0.638038
75%        1.054136
max       17.661098
Name: next_day_return, dtype: float64

翌日プラスになる確率: 37.4%
翌日マイナスになる確率: 62.6%


### 4.1 Active: 儲かった vs 損した の比較

In [20]:
# 儲かった銘柄 vs 損した銘柄
df_active_win = df_active[df_active['next_day_return'] > 0].copy()
df_active_lose = df_active[df_active['next_day_return'] <= 0].copy()

print(f"儲かった: {len(df_active_win)} ({len(df_active_win)/len(df_active)*100:.1f}%)")
print(f"損した: {len(df_active_lose)} ({len(df_active_lose)/len(df_active)*100:.1f}%)")

# テクニカル指標の平均値を比較
indicators = ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'active_score']

comparison = pd.DataFrame({
    '指標': indicators,
    '儲かった平均': [df_active_win[ind].mean() for ind in indicators],
    '損した平均': [df_active_lose[ind].mean() for ind in indicators],
})
comparison['差分'] = comparison['儲かった平均'] - comparison['損した平均']
comparison['差分%'] = (comparison['差分'] / comparison['損した平均'].abs() * 100)

print("\n--- Active: 儲かった vs 損した の指標比較 ---")
display(comparison.style.set_properties(**{'text-align': 'left'}))

儲かった: 128 (37.4%)
損した: 214 (62.6%)

--- Active: 儲かった vs 損した の指標比較 ---


,指標,儲かった平均,損した平均,差分,差分%
0,change_pct,-1.328359,-0.368411,-0.959948,-260.564315
1,atr14_pct,6.393594,6.940047,-0.546453,-7.873909
2,rsi14,45.507266,48.939813,-3.432547,-7.013814
3,vol_ratio,135.277187,157.700748,-22.423560,-14.219058
4,active_score,70.976562,73.439252,-2.462690,-3.353370


### 4.2 Active: 各指標と翌日リターンの相関分析

In [21]:
# 相関係数を計算
correlation_indicators = ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'active_score']
correlations = {}

for ind in correlation_indicators:
    corr = df_active[[ind, 'next_day_return']].corr().iloc[0, 1]
    correlations[ind] = corr

corr_df = pd.DataFrame({
    '指標': list(correlations.keys()),
    '相関係数': list(correlations.values())
}).sort_values('相関係数', key=abs, ascending=False)

print("--- Active: 各指標と翌日リターンの相関係数 ---")
display(corr_df.style.set_properties(**{'text-align': 'left'}))

print("\n解釈:")
print("- 相関係数 > 0.1: その指標が高いほど翌日儲かる")
print("- 相関係数 < -0.1: その指標が高いほど翌日損する")
print("- |相関係数| < 0.1: ほぼ関係ない")

--- Active: 各指標と翌日リターンの相関係数 ---


,指標,相関係数
3,vol_ratio,-0.180486
2,rsi14,-0.163563
0,change_pct,-0.142001
4,active_score,-0.123895
1,atr14_pct,-0.039495



解釈:
- 相関係数 > 0.1: その指標が高いほど翌日儲かる
- 相関係数 < -0.1: その指標が高いほど翌日損する
- |相関係数| < 0.1: ほぼ関係ない


## 5. 結論と改善提案

In [22]:
print("="*60)
print("改善提案サマリー")
print("="*60)

print("\n【問題点】")
print("1. overall_rating が機能していない可能性")
print("   → 全て '中立' なら、40点の配点が無駄")
print("\n2. 現状のスコアリングが翌日リターンと相関していない可能性")
print("   → スコアが高い = 儲かる、になっていない")
print("\n3. Entry/Active の候補数が少なすぎる")
print("   → 100銘柄中5-10銘柄では、3700銘柄でも200銘柄以下")

print("\n【改善の優先順位】")
print("\n🥇 優先度1: overall_rating を削除")
print("   理由: 情報量が少なく、40点の配点が無駄")
print("   対策: MA, RSI を直接スコアリングに使う")

print("\n🥈 優先度2: 相関が高い指標に配点を集中")
print("   理由: 翌日リターンと相関が高い指標を重視すべき")
print("   対策: 相関分析結果に基づいて配点を再設計")

print("\n🥉 優先度3: フィルタ条件を緩和")
print("   理由: 候補数が少なすぎる")
print("   対策: 価格帯・ATR・change_pct の範囲を広げる")

print("\n" + "="*60)

改善提案サマリー

【問題点】
1. overall_rating が機能していない可能性
   → 全て '中立' なら、40点の配点が無駄

2. 現状のスコアリングが翌日リターンと相関していない可能性
   → スコアが高い = 儲かる、になっていない

3. Entry/Active の候補数が少なすぎる
   → 100銘柄中5-10銘柄では、3700銘柄でも200銘柄以下

【改善の優先順位】

🥇 優先度1: overall_rating を削除
   理由: 情報量が少なく、40点の配点が無駄
   対策: MA, RSI を直接スコアリングに使う

🥈 優先度2: 相関が高い指標に配点を集中
   理由: 翌日リターンと相関が高い指標を重視すべき
   対策: 相関分析結果に基づいて配点を再設計

🥉 優先度3: フィルタ条件を緩和
   理由: 候補数が少なすぎる
   対策: 価格帯・ATR・change_pct の範囲を広げる



## 6. 結果コピー用（チャットに貼り付け用）

In [23]:
print("=" * 80)
print("チャット貼り付け用結果サマリー")
print("=" * 80)

print("\n### Entry戦略")
print(f"- 候補数: {len(df_entry)}")
print(f"- 翌日勝率: {(df_entry['next_day_return'] > 0).sum() / len(df_entry) * 100:.1f}%")
print(f"- 平均リターン: {df_entry['next_day_return'].mean():.2f}%")

print("\n### Entry: 儲かった vs 損した")
print("指標 | 儲かった平均 | 損した平均 | 差分")
print("-" * 60)
indicators = ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'entry_score']
for ind in indicators:
    win_mean = df_entry_win[ind].mean()
    lose_mean = df_entry_lose[ind].mean()
    diff = win_mean - lose_mean
    print(f"{ind:15s} | {win_mean:12.2f} | {lose_mean:11.2f} | {diff:+7.2f}")

print("\n### Entry: 各指標と翌日リターンの相関係数")
print("指標 | 相関係数")
print("-" * 40)
for ind in ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'entry_score']:
    corr = df_entry[[ind, 'next_day_return']].corr().iloc[0, 1]
    print(f"{ind:15s} | {corr:+8.4f}")

print("\n### Entry: overall_rating別の勝率")
for rating in df_entry['overall_rating'].unique():
    subset = df_entry[df_entry['overall_rating'] == rating]
    win_rate = (subset['next_day_return'] > 0).sum() / len(subset) * 100
    print(f"{rating}: {win_rate:.1f}% (n={len(subset)})")

print("\n" + "=" * 80)

print("\n### Active戦略")
print(f"- 候補数: {len(df_active)}")
print(f"- 翌日勝率: {(df_active['next_day_return'] > 0).sum() / len(df_active) * 100:.1f}%")
print(f"- 平均リターン: {df_active['next_day_return'].mean():.2f}%")

print("\n### Active: 儲かった vs 損した")
print("指標 | 儲かった平均 | 損した平均 | 差分")
print("-" * 60)
indicators = ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'active_score']
for ind in indicators:
    win_mean = df_active_win[ind].mean()
    lose_mean = df_active_lose[ind].mean()
    diff = win_mean - lose_mean
    print(f"{ind:15s} | {win_mean:12.2f} | {lose_mean:11.2f} | {diff:+7.2f}")

print("\n### Active: 各指標と翌日リターンの相関係数")
print("指標 | 相関係数")
print("-" * 40)
for ind in ['change_pct', 'atr14_pct', 'rsi14', 'vol_ratio', 'active_score']:
    corr = df_active[[ind, 'next_day_return']].corr().iloc[0, 1]
    print(f"{ind:15s} | {corr:+8.4f}")

print("\n" + "=" * 80)

チャット貼り付け用結果サマリー

### Entry戦略
- 候補数: 203
- 翌日勝率: 39.9%
- 平均リターン: -0.09%

### Entry: 儲かった vs 損した
指標 | 儲かった平均 | 損した平均 | 差分
------------------------------------------------------------
change_pct      |        -0.21 |        0.04 |   -0.25
atr14_pct       |         2.07 |        2.04 |   +0.03
rsi14           |        46.62 |       56.72 |  -10.10
vol_ratio       |       101.46 |      104.17 |   -2.71
entry_score     |        59.10 |       58.13 |   +0.97

### Entry: 各指標と翌日リターンの相関係数
指標 | 相関係数
----------------------------------------
change_pct      |  +0.0015
atr14_pct       |  +0.0799
rsi14           |  -0.1160
vol_ratio       |  -0.0815
entry_score     |  +0.0147

### Entry: overall_rating別の勝率
中立: 39.9% (n=203)


### Active戦略
- 候補数: 342
- 翌日勝率: 37.4%
- 平均リターン: -0.76%

### Active: 儲かった vs 損した
指標 | 儲かった平均 | 損した平均 | 差分
------------------------------------------------------------
change_pct      |        -1.33 |       -0.37 |   -0.96
atr14_pct       |         6.39 |        6.94 |   -0.55
rsi